## Notebook-1: Custom Function Creation Based experiment on Huggingface Dataset

### Notebook Contents and Objective on Notebook-1 and Notebook-2:

#### Notebook-1: Custom Function Based experiment
This notebook will cover Important part to prepare Donut Dataset for all task but here only we experimented on DOCVQA datset

- 1. Detail experiment of Donut ground truth and gt_parse structure formation and create add_ground_truth custom function
- 2. create add_tokens custom function and its embedding
- 3. json2token experiment and create custom function to make proper prmpt sequence
- 4. tokenize and PyTorch tensor conversion of input image pizel, decoder input ids and labels which is the masking clone of decodder input ids from huggingface dataset
- 5. Convert huggingface dataset to PyTorch Dataset also taking only three variables (input image pixel values, decoder input ids and labels mask tensor) which will be mainly required for our DONUT model and then these tensor will be processed for model building

#### Notebook-2: OOPs Based experiment
This notebook will be mainly focused on creating Donut Dataset class from PyTorch and it will combine all the above task inside one class using OOPs
- 1. Creating custom function for ground truth task i.e. 'gt_parses'
- 2. Creating DonutDataset class which consists of all rest task such as add_tokens, json2tokens, tokenize and tensor formation which will directly convert huggingface dataset to Pytorch dataset and will return input image pixel values, decoder input ids and labels mask tensors for further Donut model buiding task using PyTorch

### Importing necessary Dependencies

In [1]:
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00


In [2]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import os
import json
from google.colab import data_table
import datasets
from datasets import Dataset
import json
import re
import random
from typing import Any, List, Tuple

from PIL import Image
import torch
print("CUDA available:", torch.cuda.is_available())
# from donut import DonutModel
import transformers
from transformers import DonutProcessor, VisionEncoderDecoderConfig, VisionEncoderDecoderModel, BartConfig
from torch.utils.data import DataLoader


CUDA available: False


In [5]:
model_checkpoint = "naver-clova-ix/donut-base"
max_length = 128
image_size = [1280, 960]


config = VisionEncoderDecoderConfig.from_pretrained(model_checkpoint)
config.encoder.image_size = image_size # (height, width)
# update max_length of the decoder (for generation)
config.decoder.max_length = max_length

processor = DonutProcessor.from_pretrained(model_checkpoint)
model = VisionEncoderDecoderModel.from_pretrained(model_checkpoint, config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


## S-1: Intial Steps to till Ground Truth

In [6]:
train_path_main = '/content/drive/MyDrive/docVqa_dataset/train/train_v1.0.json'
# Open the JSON file
with open(train_path_main, 'r') as file:
  # Load the contents of the file into a dictionary
  data_train = json.load(file)

## Convert 'data' list into dataframe by passing into the dataframe
df_train = pd.DataFrame(data_train['data'])

## Making subset of the training dataframe
df_sub = df_train.iloc[:10]

base_train_path = '/content/drive/MyDrive/docVqa_dataset/train'

# Define a function to join the base path with the image path
def join_paths(image_path):
    return os.path.join(base_train_path, image_path)

# Apply the function to create a new column with the full image path
df_sub['full_path_image'] = df_sub['image'].apply(join_paths)


def extract_ocr_info(df, base_train_dir):

  dir_list = os.listdir(base_train_dir)
  ocr_base_dir = os.path.join(base_train_dir, dir_list[1])

  # Create empty lists for bounding boxes and word lists
  bbox_list = []
  word_list = []

  # Iterate over each row in the DataFrame
  for _, row in df.iterrows():

    # Construct the ocr_path for the row
    ocr_path = os.path.join(ocr_base_dir, row['ucsf_document_id'] + '_' + row['ucsf_document_page_no'] + '.json')

    # Open the json file and load the data
    with open(ocr_path, 'r') as f:
      ocr_json = json.load(f)

    # Extract the bounding boxes and word lists from the json data
    bbox_row = []
    word_row = []
    for i in ocr_json['recognitionResults'][0]['lines']:
      for j in i['words']:
        bbox_row.append(j['boundingBox'])
        word_row.append(j['text'])

    # Append the bbox and word lists to the global lists
    bbox_list.append(bbox_row)
    word_list.append(word_row)
  # Create new columns in the DataFrame to store the bbox and words lists
  df['bounding_boxes'] = bbox_list
  df['word_list'] = word_list

  df.drop(['questionId', 'image','ucsf_document_id', 'ucsf_document_page_no'], axis=1, inplace=True)

  # Return the updated DataFrame
  return df


df_sub_ocr = extract_ocr_info(df_sub, base_train_path)

## Convert dataframe into dataset object
hf_sub_ocr = Dataset.from_pandas(df_sub_ocr)


<ipython-input-6-f4858d3802c0>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub['full_path_image'] = df_sub['image'].apply(join_paths)
<ipython-input-6-f4858d3802c0>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bounding_boxes'] = bbox_list
<ipython-input-6-f4858d3802c0>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [7]:
hf_sub_ocr    ## Row column check

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list'],
    num_rows: 10
})

In [8]:
hf_sub_ocr.column_names     ## printing number of columns

['question',
 'docId',
 'answers',
 'data_split',
 'full_path_image',
 'bounding_boxes',
 'word_list']

In [9]:
hf_sub_ocr.features   ## checking features details

{'question': Value(dtype='string', id=None),
 'docId': Value(dtype='int64', id=None),
 'answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'data_split': Value(dtype='string', id=None),
 'full_path_image': Value(dtype='string', id=None),
 'bounding_boxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'word_list': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [10]:
len(hf_sub_ocr)   ## total number of rows present

10

## S-1.1: ground_truth experiment & gt_parse prompt preparation and add_ground_truth custom function creation


In [11]:
hf_sub_ocr['question'][0]   ## checking 1st row question

'what is the date mentioned in this letter?'

In [12]:
## iterate over all questions and its data type

for ques in hf_sub_ocr['question']:
  print(ques, "|||||",type(ques))     # <class 'str'>

what is the date mentioned in this letter? ||||| <class 'str'>
what is the contact person name mentioned in letter? ||||| <class 'str'>
Which corporation's letterhead is this? ||||| <class 'str'>
Who is in  cc in this letter? ||||| <class 'str'>
what is the subject of  this letter? ||||| <class 'str'>
What is the number at the bottom of the page, in bold? ||||| <class 'str'>
How many points are there in modifications to readout instrumentation ||||| <class 'str'>
what is the date in the letter ||||| <class 'str'>
To whom this is addressed ||||| <class 'str'>
Who sent the letter? ||||| <class 'str'>


## .replace('"', '\\"')
Certainly! Let's consider an example to understand the code `questions = questions.replace('"', '\\"')`:

```python
questions = 'What is your "name"?"'
questions = questions.replace('"', '\\"')
print(questions)
```

In this example, we have a string `questions` that contains a question: "What is your "name"?" We want to replace the double quote character (`"`) within the string with a backslash followed by a double quote (`\"`).

The line `questions = questions.replace('"', '\\"')` performs the replacement. It searches for all occurrences of the double quote (`"`) character in the `questions` string and replaces them with `\"`. The resulting modified string is assigned back to the variable `questions`.

When we print the value of `questions`, the output will be:
```
What is your \"name\"?
```

The original double quotes within the string are replaced with `\"`, where the backslash `\` serves as an escape character to indicate that the following double quote should be treated as part of the string, rather than as a closing delimiter.

So, the purpose of the code is to modify the original string by escaping double quotes with a backslash, allowing them to be properly interpreted within the string.

In [13]:
hf_sub_ocr['answers'][0]    ## checking 1st row question

['1/8/93']

In [14]:
## iterate over all answers and its data type
for ans in hf_sub_ocr['answers']:
  print(ans, "|||||",type(ans))   # <class 'list'>

['1/8/93'] ||||| <class 'list'>
['P. Carter', 'p. carter'] ||||| <class 'list'>
['Brown & Williamson Tobacco Corporation'] ||||| <class 'list'>
['T.F. Riehl'] ||||| <class 'list'>
['Review of existing Brainstorming Ideas/483'] ||||| <class 'list'>
['499150498'] ||||| <class 'list'>
['5.', '5'] ||||| <class 'list'>
['June 11, 1990', 'June 11,1990'] ||||| <class 'list'>
['Dr.K.S.Houghton'] ||||| <class 'list'>
['Ted Sanders'] ||||| <class 'list'>


In [15]:
ans1_before = hf_sub_ocr['answers'][2]
ans1_before

['Brown & Williamson Tobacco Corporation']

In [16]:
ans1_after = ans1_before[0] .replace('"', '\"')
ans1_after

'Brown & Williamson Tobacco Corporation'

### Note:
Certainly! Let's consider an example to understand the code `ans = answ.replace('"', '\"')`: .replace('"', '\"')

```python
answ = 'Hello "world"'
ans = answ.replace('"', '\"')
print(ans)
```

In this example, we have a string `answ` that contains the phrase "Hello "world"". We want to replace the double quote character (`"`) within the string with a backslash followed by a double quote (`\"`).

The line `ans = answ.replace('"', '\"')` performs the replacement. It searches for all occurrences of the double quote (`"`) character in the `answ` string and replaces them with `\"`. The resulting modified string is assigned to the variable `ans`.

When we print the value of `ans`, the output will be:
```
Hello \"world\"
```

The original double quotes are replaced with `\"`, where the backslash `\` serves as an escape character to indicate that the following double quote should be treated as part of the string, rather than as a closing delimiter.

So, the purpose of the code is to modify the original string by escaping double quotes with a backslash, allowing them to be properly interpreted within the string.

### Note structure of the ground truth for different Task:
custom function creation for gt_parse for all kind of task
```
GT for QA Task:

## For Single Question and Single Answer
'{"gt_parses": [{"question" : "...........", "answer" : "........"}]}'

or
## For Multiple Question and Multiple Answer
'{"gt_parses": [{"question1" : "...........", "answer1" : "........"},
{"question2" : "...........", "answer2" : "........"}]}'

or
## For Single Question Multiple Answer
'{"gt_parses": [{"question" : "...........", "answer1" : "........",
                                    "answer2" : "........", "answer3" : "........"..so on}]}'
```

```
GT for Doc Classification Task:

'{"gt_parse": {"class" : "class_name"}}'

```

```
Sample helping code:

template = '{"gt_parse": {"class" : '

def add_ground_truth(hf_examples):

  ground_truths = []

  for label in hf_examples['label']:
    ground_truths.append(template + '"' + id2label[label] + '"' + "}}")

  hf_examples['ground_truth'] = ground_truths

  return hf_examples

## Mapping with the
hf_dataset_gt = hf_dataset.map(add_ground_truth, batched=True)

```


```
GT for Doc Parsing task:
{"gt_parses": [{"company": "ADVANCO COMPANY",
"date": "17/01/2018",
"address": "NO 1&3, JALAN WANGSA DELIMA 12, WANGSA LINK, WANGSA MAJU, 53300 KUALA LUMPUR",
"total": "7.00"}]}

```
```
Sample helping code:

parse = {
  "company": "ADVANCO COMPANY",
  "date": "17/01/2018",
  "address": "NO 1&3, JALAN WANGSA DELIMA 12, WANGSA LINK, WANGSA MAJU, 53300 KUALA LUMPUR",
  "total": "7.00"
}

print(parse, type(parse))

gt_example = '{"gt_parses": [{'
# print(gt_example)

for key, value in parse.items():
  value = value.replace('"', '\"')
  gt_example += '"' + key + '": "' + value + '", '
  # print(gt_example)
gt_example = gt_example.rstrip(', ')
gt_example += '}]}'
print(gt_example)

```

```
Sample Custom Functom:
def add_ground_truth(hf_examples):

  keys = hf_examples.keys()  # Get the keys of the example dictionary

  ground_truths = []    # Creating an empty list for ground truth prompts

  for example in hf_examples:
    ground_truth_example = '{"gt_parses": [{'  # Start of the ground truth prompt string

    # Iterate over the keys and values of each example
    for key, value in example.items():
      value = value.replace('"', '\"')   # Replacement of " from each value
      ground_truth_example += '"' + key + '": "' + value + '", '  # Append key-value pairs to the ground truth prompt

    ground_truth_example = ground_truth_example.rstrip(', ')  # Remove the trailing comma and space
    ground_truth_example += ']}]'  # End of the ground truth prompt string

    ground_truths.append(ground_truth_example)  # Add the ground truth prompt to the list

  hf_examples['ground_truth'] = ground_truths  # Add the ground truth prompts to the example dictionary

  return hf_examples


hf_dataset_gt = hf_dataset.map(add_ground_truth, batched=True)

print(hf_dataset_gt['ground_truth'][0])
```

In [19]:
# Creating custom function for DOCVQA task

def add_ground_truth(hf_examples):
  images = hf_examples['full_path_image']
  questions = hf_examples['question']
  answers = hf_examples['answers']

  ground_truths = []    # Creating empty list
  for image,question, answer in zip(images,questions, answers):
    # we need to escape " characters appearing in the query and/or answer
    question = question.replace("\\", "") ## this was just one corrupt example (index 91 of training set)
    question = re.sub(' +', ' ', question)
    question = question.replace('"', '\\"')   # replacement of " from question
    # let's create the ground truth string
    ground_truth_example = '{"gt_parses": ['
    for idx, answ in enumerate(answer):
      # ans = answ.replace('"', '\"')   # replacement of " from answer
      answ = answ.replace("\\", "")
      ans = answ.replace('"', '\\"')
      ground_truth_example += '{"question" : "' + question + '", "answer" : "' + ans + '"}'

      # add comma for more than one element present in the answer list
      if idx != len(answer) - 1:   # when current index is not equal to last index
        ground_truth_example += ', '
    ground_truth_example += ']}'
    ground_truths.append(ground_truth_example)    ## appending ground_truths list for every row

  hf_examples['ground_truth'] = ground_truths

  return hf_examples

In [20]:
hf_sub_ocr_gt = hf_sub_ocr.map(add_ground_truth, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Note: Why 'batched = True' parameter?
- When 'batched = True', it suggests that the code or function is designed to handle data in batches. The exact behavior and implications of this parameter can vary depending on the specific code or function being used.

- Typically, when processing data in batches, it means that instead of processing individual examples one by one, the code or function operates on a group of examples simultaneously. This can improve computational efficiency by leveraging parallel processing and optimizing memory usage.

In [21]:
hf_sub_ocr_gt

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth'],
    num_rows: 10
})

In [22]:
print(hf_sub_ocr_gt['ground_truth'][2], '|||||' , type(hf_sub_ocr_gt['ground_truth'][2]))

{"gt_parses": [{"question" : "Which corporation's letterhead is this?", "answer" : "Brown & Williamson Tobacco Corporation"}]} ||||| <class 'str'>


## S-2: Creating add_tokens custom function for all type of task

In [23]:
len(processor.tokenizer)

57525

In [24]:
## Custom function to add new starting and ending prompt tokens and embed it
new_special_tokens_list = []
# list with the newly added tokens. The purpose of this list is to keep track of the tokens that were added to the tokenizer.

def add_tokens(list_of_tokens: List[str]):
  """
  Add tokens to tokenizer and resize the token embeddings
  """
  newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
  if newly_added_num > 0:
    model.decoder.resize_token_embeddings(len(processor.tokenizer))
    new_special_tokens_list.extend(list_of_tokens)


additional_tokens = ["<yes/>", "<no/>"]

add_tokens(additional_tokens)

In [25]:
len(processor.tokenizer)

57527

## S-3: json2token experiment and custom function creation

#### JSON and JSON2TOKEN Sample Experiment:

In [26]:
## JSON object of dict type
json_str_dic = '{"name": "John Doe", "age": 30, "city": "New York"}'
print(type(json_str_dic), json_str_dic)

json_obj_dic = json.loads(json_str_dic)
print(type(json_obj_dic), json_obj_dic)

<class 'str'> {"name": "John Doe", "age": 30, "city": "New York"}
<class 'dict'> {'name': 'John Doe', 'age': 30, 'city': 'New York'}


In [27]:
print(json_obj_dic.keys())    ## returns name of keys
print(len(json_obj_dic))   ## returns number of keys

dict_keys(['name', 'age', 'city'])
3


In [28]:
## 'in' will check whether string ia present in the keys of the json object or not
print('New York' in json_obj_dic)
print('name' in json_obj_dic)

print("text_sequence" in json_obj_dic)

False
True
False


In [29]:
## Formation of Start and end token & creation of prompt:

nw_tok = []
op = ''
keys = json_obj_dic.keys()
for k in keys:
  print(k, type(k))   # collecting the keys from the gt_parses dictionary
  # print('--------------------------------')
  # print(json_obj_dic[k], type(json_obj_dic[k]))   # collecting the respective values from the dictionary
  # print('--------------------------------')

  # task-1 : adding start and end token into a new list which will later add into the processor.tokenizer class
  nw_tok.append(fr"<s_{k}>") if fr"<s_{k}>" not in nw_tok else None   # forming start token for different task using f string
  nw_tok.append(fr"</s_{k}>") if fr"</s_{k}>" not in nw_tok else None   # forming end token for different task using f string

  # task-2 : formation of the accurate prompt where
  # <s_{k}> is the start token,
  # str(json_obj_dic[k]) is the adding value part which must be string type, so need convertion
  # </s_{k} is end token
  op += (fr"<s_{k}>" + str(json_obj_dic[k]) + fr"</s_{k}>")
print('--------------------------------')
print(nw_tok, type(nw_tok))   # List of newly formed tokens which will be added into processor.tokenizer.add_tokens
print('--------------------------------')
print(op, type(op))   # finally our prompt will be look like this

name <class 'str'>
age <class 'str'>
city <class 'str'>
--------------------------------
['<s_name>', '</s_name>', '<s_age>', '</s_age>', '<s_city>', '</s_city>'] <class 'list'>
--------------------------------
<s_name>John Doe</s_name><s_age>30</s_age><s_city>New York</s_city> <class 'str'>


#### Note : Difference between .append() and .extend()
The `append()` and `extend()` methods in Python are used to add elements to a list, but they have different behaviors:

1. `append()`: The `append()` method is used to add a single element to the end of a list. It modifies the original list by adding the new element as a single item.

   Example:
   ```python
   my_list = [1, 2, 3]
   my_list.append(4)
   print(my_list)  # Output: [1, 2, 3, 4]
   ```

   In this example, `append(4)` adds the element `4` as a single item to the end of the list `my_list`. The resulting list becomes `[1, 2, 3, 4]`.

2. `extend()`: The `extend()` method is used to add multiple elements to the end of a list. It takes an iterable (such as a list, tuple, or string) as an argument and appends each item individually to the original list.

   Example:
   ```python
   my_list = [1, 2, 3]
   my_list.extend([4, 5, 6])
   print(my_list)  # Output: [1, 2, 3, 4, 5, 6]
   ```

   In this example, `extend([4, 5, 6])` appends each element in the iterable `[4, 5, 6]` individually to the end of `my_list`. The resulting list becomes `[1, 2, 3, 4, 5, 6]`.

   Note that `extend()` can also be used with other iterables like strings:

   ```python
   my_list = [1, 2, 3]
   my_list.extend("hello")
   print(my_list)  # Output: [1, 2, 3, 'h', 'e', 'l', 'l', 'o']
   ```

   Here, each character of the string `"hello"` is added individually to the list, resulting in `[1, 2, 3, 'h', 'e', 'l', 'l', 'o']`.

In summary, `append()` adds a single item to the end of a list, while `extend()` adds multiple items from an iterable to the end of a list, with each item treated individually.

In [30]:
## JSON object of list type
json_str_list = '[{"name": "John Doe", "age": 30}, {"name": "Jane Smith", "age": 25}]'
print(type(json_str_list), json_str_list)

# Convert string to json list
json_obj_list = json.loads(json_str_list)
print(type(json_obj_list), json_obj_list)
print(json_obj_list)  # List of Dictionary

<class 'str'> [{"name": "John Doe", "age": 30}, {"name": "Jane Smith", "age": 25}]
<class 'list'> [{'name': 'John Doe', 'age': 30}, {'name': 'Jane Smith', 'age': 25}]
[{'name': 'John Doe', 'age': 30}, {'name': 'Jane Smith', 'age': 25}]


In [31]:
for item in json_obj_list:    # iterating over list
  print(item, type(item))

{'name': 'John Doe', 'age': 30} <class 'dict'>
{'name': 'Jane Smith', 'age': 25} <class 'dict'>


In [32]:
[item for item in json_obj_list]

[{'name': 'John Doe', 'age': 30}, {'name': 'Jane Smith', 'age': 25}]

In [33]:
[str(item) for item in json_obj_list]   # convert dictionary list to string list

["{'name': 'John Doe', 'age': 30}", "{'name': 'Jane Smith', 'age': 25}"]

In [34]:
## S-1: If in a page more than one dictionary will present inside gt_parses i.e when list of dictionary will present
# S-2: iterate and isolate each dictionary
# S-3: Convert each dictionary into string type
# S-4: Create list of two or more strings
# S-5: Join each string element of dictionary by <sep/>
# r before the string indicates represents a raw string literal

r"<sep/>".join([str(item) for item in json_obj_list])

"{'name': 'John Doe', 'age': 30}<sep/>{'name': 'Jane Smith', 'age': 25}"

### Note: r or raw string literals
- A raw string literal is a string that preserves the literal value of each character within the string, including escape characters. It is denoted by the prefix r or R.

- The use of a raw string literal is particularly useful when you have special characters or escape sequences within the string that you want to be treated as literal characters rather than being interpreted. It simplifies the handling of backslashes and escape characters, making the code more readable and less error-prone.

Certainly! Let's compare a raw string and a normal string with an example:

```python
normal_string = "C:\path\to\file"
raw_string = r"C:\path\to\file"

print(normal_string)
print(raw_string)
```

Output:
```
C:\path      o
ile
C:\path\to\file
```

In this example, we have a string representing a file path. The first string (`normal_string`) is a normal string, while the second string (`raw_string`) is a raw string.

Notice that in the `normal_string`, the backslashes (`\`) are interpreted as escape characters. However, in the `raw_string`, the backslashes are treated literally as part of the string.

In the output, you can observe the difference between the two strings. The `normal_string` ends up with unexpected characters due to the interpretation of escape sequences. On the other hand, the `raw_string` maintains the exact characters, including the backslashes, as it is defined with a raw string literal.

This demonstrates that raw strings are useful when you want to preserve backslashes and treat them as literal characters, without having them interpreted as escape sequences.

In [35]:
hf_sub_ocr_gt['ground_truth'][1], type(hf_sub_ocr_gt['ground_truth'][1])    # str

('{"gt_parses": [{"question" : "what is the contact person name mentioned in letter?", "answer" : "P. Carter"}, {"question" : "what is the contact person name mentioned in letter?", "answer" : "p. carter"}]}',
 str)

In [36]:
## Convert this str type to dict type
sample_1_dict = json.loads(hf_sub_ocr_gt['ground_truth'][1])
sample_1_dict, type(sample_1_dict)    # dict

({'gt_parses': [{'question': 'what is the contact person name mentioned in letter?',
    'answer': 'P. Carter'},
   {'question': 'what is the contact person name mentioned in letter?',
    'answer': 'p. carter'}]},
 dict)

In [37]:
## sample check of the gt_parses
if "gt_parses" in sample_1_dict:    # to check whether key present in the dictionary or not
  print('Yes Present')
  print('********************************************')
  assert isinstance(sample_1_dict["gt_parses"], list)    # verifies that the value associated with the key "gt_parses" in the ground_truth dictionary is of type list

  gt_value = sample_1_dict['gt_parses']   ## Storing value of gt_parses key
  print(gt_value, type(gt_value))   ## <class 'list'>
  print('********************************************')
  for val in gt_value:
    print(val, type(val))   # <class 'dict'>

Yes Present
********************************************
[{'question': 'what is the contact person name mentioned in letter?', 'answer': 'P. Carter'}, {'question': 'what is the contact person name mentioned in letter?', 'answer': 'p. carter'}] <class 'list'>
********************************************
{'question': 'what is the contact person name mentioned in letter?', 'answer': 'P. Carter'} <class 'dict'>
{'question': 'what is the contact person name mentioned in letter?', 'answer': 'p. carter'} <class 'dict'>


In [38]:
sample_2 = hf_sub_ocr_gt['ground_truth'][2]
sample_2, type(sample_2)    ## gt_parse is a string type data

('{"gt_parses": [{"question" : "Which corporation\'s letterhead is this?", "answer" : "Brown & Williamson Tobacco Corporation"}]}',
 str)

In [39]:
## Convert this str type to dict type
sample_2_dict = json.loads(sample_2)
sample_2_dict, type(sample_2_dict)    # dict

({'gt_parses': [{'question': "Which corporation's letterhead is this?",
    'answer': 'Brown & Williamson Tobacco Corporation'}]},
 dict)

In [40]:
## sample check of the gt_parses
if "gt_parses" in sample_2_dict:    # to check whether key present in the dictionary or not
  print('Yes Present')
  assert isinstance(sample_2_dict["gt_parses"], list)    # verifies that the value associated with the key "gt_parses" in the ground_truth dictionary is of type list

  gt_value = sample_2_dict['gt_parses']   ## Storing value of gt_parses key
  print(gt_value, type(gt_value))   ## <class 'list'>
  print('********************************************')
  for val in gt_value:
    print(val, type(val))   # <class 'dict'>


Yes Present
[{'question': "Which corporation's letterhead is this?", 'answer': 'Brown & Williamson Tobacco Corporation'}] <class 'list'>
********************************************
{'question': "Which corporation's letterhead is this?", 'answer': 'Brown & Williamson Tobacco Corporation'} <class 'dict'>


In [41]:
## Checking different tokens from processor.tokenizer class
processor.tokenizer.cls_token, processor.tokenizer.eos_token, processor.tokenizer.mask_token, processor.tokenizer.bos_token, processor.tokenizer.pad_token

('<s>', '</s>', '<mask>', '<s>', '<pad>')

### Creating JSON2TOKEN Custom function for all type of Task

In [42]:
## function converts an ordered JSON object into a token sequence
## For all type of task we can use this function

# def json2token(json_obj: Any, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True):
def json2token(json_obj: Any, sort_json_key: bool = True):

  """
  Convert an ordered JSON object into a token sequence
  'obj' (the JSON object to convert)

  'update_special_tokens_for_json_key' is set to True, the code adds special tokens to
  the new_special_tokens list for each JSON key encountered during tokenization.
  Note: The purpose of these special tokens is to provide additional context or information about the specific JSON key during tokenization.
  If 'update_special_tokens_for_json_key' is set to False, the code skips updating special tokens for JSON keys.

  'sort_json_key' (a boolean indicating whether to sort the JSON keys)

  """
  ## Prompt creation from json dict object

  if type(json_obj) == dict:   # checking if json object is dictionary type or not

    ## special case handling for JSON objects that have a single key-value pair and where the key is "text_sequence"
    if len(json_obj) == 1 and "text_sequence" in json_obj:
        return json_obj["text_sequence"]  # returns the value of "text_sequence" key

    else:   # if more than one keys are present also not contains "text_sequence" key
      output = ""   # initializes an empty string output to store the token sequence

      if sort_json_key:   # sort_json_key is True, it sorts the dictionary keys in reverse order
          keys = sorted(json_obj.keys(), reverse=True)
      else:               # otherwise it will store original keys
          keys = json_obj.keys()

      # iterates over keys
      for k in keys:

        # if update_special_tokens_for_json_key:    # if the value is True
        #     new_special_tokens.append(fr"<s_{k}>") if fr"<s_{k}>" not in new_special_tokens else None
        #     new_special_tokens.append(fr"</s_{k}>") if fr"</s_{k}>" not in new_special_tokens else None

        add_tokens([fr"<s_{k}>", fr"</s_{k}>"])   # adding special tokens

        # It recursively calls 'json2token' to process the corresponding value json_obj[k] and concatenates the result with the special tokens.
        output += (
            fr"<s_{k}>"
            + json2token(json_obj[k], sort_json_key)      # here json_obj[k] is string and value part of the json key
            + fr"</s_{k}>"
        )
      return output
      ## returns the accumulated output string/prompt: e.g <s_name>John Doe</s_name><s_age>30</s_age><s_city>New York</s_city>

  elif type(json_obj) == list:    # if json_obj contains more than one dictionary e.g [{'name': 'John Doe', 'age': 30}, {'name': 'Jane Smith', 'age': 25}]
    return r"<sep/>".join(
        # [json2token(item, update_special_tokens_for_json_key, sort_json_key) for item in json_obj]        # here item is string
        [json2token(item, sort_json_key) for item in json_obj]        # here item is list of dictionary
    )
  ## returns the string of two dictionary separated by <sep/> e.g {'name': 'John Doe', 'age': 30}<sep/>{'name': 'Jane Smith', 'age': 25}
  # which will again pass through json2token and will form the prompt structure
  # e.g : <s_name>John Doe</s_name><s_age>30</s_age><sep/><s_name>Jane Smith</s_name><s_age>25</s_age>

## last part json_obj does not match the previous conditions of being a dictionary or a list. It serves as a fallback option for handling other types of objects.
  else:
    # excluded special tokens for now
    json_obj = str(json_obj)      # converts the json_obj into its string

    if f"<{json_obj}/>" in new_special_tokens_list:
      json_obj = f"<{json_obj}/>"  # for categorical special tokens
    return json_obj


In [43]:
## process the dataset for all type of task
def preprocess_documents_for_donut(hf_examples):

  # convert a string representation of ground_truth a JSON object
  gt_json_obj = json.loads(hf_examples['ground_truth'])

  if "gt_parses" in gt_json_obj:
    assert isinstance(gt_json_obj["gt_parses"], list)
    gt_jsons_list = gt_json_obj['gt_parses']    ## Stores the list of gt_parses value as gt_parses is the only key
    # gt_prompt = processor.tokenizer.cls_token + json2token(gt_jsons_list) + processor.tokenizer.eos_token

  else:
    assert "gt_parse" in gt_json_obj and isinstance(gt_json_obj["gt_parses"], dict)
    gt_jsons_list = [gt_json_obj['gt_parses']]

  ## this is for DOCVQA task
  gt_prompt = processor.tokenizer.cls_token + json2token(gt_jsons_list) + processor.tokenizer.eos_token

  ## this is for Document classification task
  # gt_prompt = processor.tokenizer.cls_token + "<s_docClass>" + json2token(gt_jsons_list) + "</s_docClass>" + processor.tokenizer.eos_token

  ## this is for Doc parsing task
  # gt_prompt = processor.tokenizer.cls_token + "<s_docParse>" + json2token(gt_jsons_list) + "</s_docParse>" + processor.tokenizer.eos_token

  hf_examples['gt_prompt'] = gt_prompt
  return hf_examples

In [44]:
hf_sub_ocr_gt_prompt = hf_sub_ocr_gt.map(preprocess_documents_for_donut)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [45]:
hf_sub_ocr_gt_prompt

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth', 'gt_prompt'],
    num_rows: 10
})

In [46]:
hf_sub_ocr_gt_prompt['gt_prompt']     ## checking all the gt_prompts

['<s><s_question>what is the date mentioned in this letter?</s_question><s_answer>1/8/93</s_answer></s>',
 '<s><s_question>what is the contact person name mentioned in letter?</s_question><s_answer>P. Carter</s_answer><sep/><s_question>what is the contact person name mentioned in letter?</s_question><s_answer>p. carter</s_answer></s>',
 "<s><s_question>Which corporation's letterhead is this?</s_question><s_answer>Brown & Williamson Tobacco Corporation</s_answer></s>",
 '<s><s_question>Who is in cc in this letter?</s_question><s_answer>T.F. Riehl</s_answer></s>',
 '<s><s_question>what is the subject of this letter?</s_question><s_answer>Review of existing Brainstorming Ideas/483</s_answer></s>',
 '<s><s_question>What is the number at the bottom of the page, in bold?</s_question><s_answer>499150498</s_answer></s>',
 '<s><s_question>How many points are there in modifications to readout instrumentation</s_question><s_answer>5.</s_answer><sep/><s_question>How many points are there in modifi

In [47]:
hf_sub_ocr_gt_prompt['gt_prompt'][0] , type(hf_sub_ocr_gt_prompt['gt_prompt'][0])   # 0th row   # str

('<s><s_question>what is the date mentioned in this letter?</s_question><s_answer>1/8/93</s_answer></s>',
 str)

In [48]:
hf_sub_ocr_gt_prompt['gt_prompt'][1] , type(hf_sub_ocr_gt_prompt['gt_prompt'][1])   # 1st row

('<s><s_question>what is the contact person name mentioned in letter?</s_question><s_answer>P. Carter</s_answer><sep/><s_question>what is the contact person name mentioned in letter?</s_question><s_answer>p. carter</s_answer></s>',
 str)

In [49]:
hf_sub_ocr_gt_prompt['gt_prompt'][2]    # 2nd row

"<s><s_question>Which corporation's letterhead is this?</s_question><s_answer>Brown & Williamson Tobacco Corporation</s_answer></s>"

## S-4: transform_and_tokenize experiment, creating tensor and custom function creation

#### Issue1: Dataset map return only list instead torch tensors
- https://discuss.huggingface.co/t/dataset-map-return-only-list-instead-torch-tensors/15767/4
-  map ignores tensor formatting while writing a cache file, so to get PyTorch tensors under the input_ids column, you need to explicitly call set_format("pt", columns=["input_ids"], output_all_columns=True) on the dataset object (after map
- When it comes to tensors, PyArrow (the storage format we use) only understands 1D arrays, so we would have to store (potentially) a significant amount of metadata to be able to restore the types after map fully. Also, a map transform can return different value types for the same column (e.g. PyTorch tensors or Python lists), which would make this process ambiguous (e.g. should we return mixed types or only one type when indexing the dataset afterwards). So for the sake of simplicity, we return Python objects by default, and for other formats, one can use with_format/with_transform
- https://huggingface.co/docs/datasets/v2.12.0/en/package_reference/main_classes#datasets.Dataset.set_format

```
e.g.
from datasets import load_dataset
from transformers import AutoTokenizer
ds = load_dataset("rotten_tomatoes", split="validation")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
ds = ds.map(lambda x: tokenizer(x['text'], truncation=True, padding=True), batched=True)
ds.set_format(type='numpy', columns=['text', 'label'])
ds.format
```
- https://huggingface.co/docs/datasets/process#flatten
- https://huggingface.co/docs/datasets/v1.4.0/processing.html

```
Format:

*** The set_format() function changes the format of a column to be compatible with some common data formats.
Specify the output you’d like in the type parameter and the columns you want to format. Formatting is applied on-the-fly.

For example, create PyTorch tensors by setting type="torch":
import torch

dataset.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])

*** The with_format() function also changes the format of a column, except it returns a new Dataset object:

dataset = dataset.with_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])

If you need to reset the dataset to its original format, use the reset_format() function:
dataset.format
>>> {'type': 'torch', 'format_kwargs': {}, 'columns': ['label'], 'output_all_columns': False}

dataset.reset_format()
dataset.format

>>> {'type': 'python', 'format_kwargs': {}, 'columns': ['idx', 'label', 'sentence1', 'sentence2'], 'output_all_columns': False}

```

#### Issue2: Colab crashing files due huge number of rows
- https://discuss.huggingface.co/t/dataset-map-hangs-on-tokenization-relatively-small-dataset/16591/3
- Instead of map, if we run for loop and apply function then there may be no file crashed issue occur
- Crashing due to crossing size limit of single row in may be each row is taking more than 15 mb space

In [50]:
img_2 = hf_sub_ocr_gt_prompt['full_path_image'][2]    ## accessing 2nd image

In [51]:
## showing the details of pixel values before squeeze
pixel_values_2 = processor(Image.open(img_2).convert("RGB"), return_tensors="pt"
).pixel_values

pixel_values_2, pixel_values_2.shape
# torch.Size([1, 3, 2560, 1920]) shape before squeeze so need to squeeze and reduce dimension where 1 is present

(tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]],
 
          [[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]],
 
          [[-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           ...,
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.],
           [-1., -1., -1.,  ..., -1., -1., -1.]]]]),
 torch.Size([1, 3, 2560, 1920]))

In [52]:
## gt prompt of 1st row sample
gpt_1 = hf_sub_ocr_gt_prompt["gt_prompt"][1]
gpt_1

'<s><s_question>what is the contact person name mentioned in letter?</s_question><s_answer>P. Carter</s_answer><sep/><s_question>what is the contact person name mentioned in letter?</s_question><s_answer>p. carter</s_answer></s>'

In [53]:
## gt prompt decoder input ids of 1st row sample
input_ids_1 = processor.tokenizer(
      gpt_1,
      add_special_tokens=False,
      max_length=max_length,
      padding="max_length",
      truncation=True,
      return_tensors="pt",
  )["input_ids"].squeeze(0)
input_ids_1, input_ids_1.shape

## before squeeze shape is torch.Size([1, 128])
## after squeeze shape is torch.Size([128])

(tensor([    0, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
         25515, 36322, 36209, 57528, 57529, 40769, 39539, 39473, 49798, 57530,
         57522, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
         25515, 36322, 36209, 57528, 57529, 55285, 39539, 17662, 39905, 57530,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [54]:
labels1 = input_ids_1.clone()   # creating clone tensor of decoder input ids

In [55]:
## Sample token tensor experiment for row 1 also details of how token extraction will work on labels tensor from decoder input ids

print(f"Labels tensor Before extraction: {labels1}")    ## labels1 tensor before extration
## Code all together
prompt_start_token_id1 = 57529      # start token ID just for example
prompt_end_token_id1 = 57530      # end token ID just for example

## returning index value 2-D tensor using of nonzero
# Collection of all start index position in label
prompt_start_token_id_index1 = (labels1 == prompt_start_token_id1).nonzero()
prompt_end_token_id_index1 = (labels1 == prompt_end_token_id1).nonzero()

## Crerating empty mask integer tensor
mask1 = torch.tensor([], dtype=torch.int)

## iterate over both start and end tensor simulteneously
for start_index1,end_index1 in zip(prompt_start_token_id_index1, prompt_end_token_id_index1):
  val_extracted1 = labels1[start_index1.squeeze()+1:end_index1.squeeze()]     ## stores all extarcted values
  mask1 = torch.cat((mask1, val_extracted1))     ## concatenating start and end index value

labels1[~torch.isin(labels1, mask1)]=-100        ## assign with ignore id
print('*******************************************************************************************')
print(f"Labels tensor After extraction: {labels1}")    ## tensor before extration

Labels tensor Before extraction: tensor([    0, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
        25515, 36322, 36209, 57528, 57529, 40769, 39539, 39473, 49798, 57530,
        57522, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
        25515, 36322, 36209, 57528, 57529, 55285, 39539, 17662, 39905, 57530,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,   

In [56]:
## For all type of task
def transform_and_tokenize(hf_examples, processor=processor, ignore_id=-100, prompt_start_token=None, prompt_end_token=None) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:

  """
      Load image from image_path of given dataset_path and convert into input_tensor and labels
      Convert gt data into input_ids (tokenized string)
      Returns:
          input_tensor : preprocessed image
          input_ids : tokenized gt_data
          labels : masked labels (model doesn't need to predict prompt and pad token)
  """
  print('*****************************************************************************************')

  add_tokens([prompt_start_token, prompt_end_token])
  prompt_start_token_id = processor.tokenizer.convert_tokens_to_ids(prompt_start_token)
  print(prompt_start_token_id)
  prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(prompt_end_token)
  print(prompt_end_token_id)

  # S-1: create input_tensor from image i.e pixel values
  try:
    pixel_values = processor(
        Image.open(hf_examples['full_path_image']).convert("RGB"), return_tensors="pt"    ## This image path part will vary as per task
    ).pixel_values

    # torch.Size([1, 3, 2560, 1920]) shape before squeeze

    input_tensor_pixel = pixel_values.squeeze()
    print(type(input_tensor_pixel),input_tensor_pixel.shape)
  except Exception as e:
    print(hf_examples)
    print(f"Error: {e}")
    return {}

  # S-2: tokenize document and creating decoder input ids
  decoder_input_ids = processor.tokenizer(
      hf_examples["gt_prompt"],
      add_special_tokens=False,
      max_length=max_length,
      padding="max_length",
      truncation=True,
      return_tensors="pt",
  )["input_ids"].squeeze(0)     #torch.Size([1, 128]) before squeeze

  print(type(decoder_input_ids),decoder_input_ids.shape)

  # S-3: Create labels which is clone tensor for extracted values from decoder input ids tensors

  labels = decoder_input_ids.clone()    ## clone tensor of decoder input ids

  ## returning index value 2-D tensor using of nonzero
  # Collection of all start index position in label

  prompt_start_token_id_index = (labels == prompt_start_token_id).nonzero
  print(prompt_start_token_id_index())
  prompt_end_token_id_index = (labels == prompt_end_token_id).nonzero
  print(prompt_end_token_id_index())

  ## Creating empty integer unmask tensor
  unmask = torch.tensor([], dtype=torch.int)

  ## iterate over both start and end tensor simulteneously
  for start_index,end_index in zip(prompt_start_token_id_index(), prompt_end_token_id_index()):
    val_extracted = labels[start_index.squeeze()+1:end_index.squeeze()]     ## stores all extarcted values
    unmask = torch.cat((unmask, val_extracted))     ## concatenating start and end index value

  labels[~torch.isin(labels, unmask)] =  ignore_id       ## assign with ignore id

  return {"input_tensor_pixel": input_tensor_pixel, "decoder_input_ids": decoder_input_ids, "labels": labels}



In [57]:
processor.image_processor.size = image_size[::-1] # should be (width, height)
processor.image_processor.do_align_long_axis = False

In [58]:
## For DOCVQA task
prompt_start_token = "<s_answer>"
prompt_end_token = "</s_answer>"

hf_sub_processed_final = hf_sub_ocr_gt_prompt.map(
    transform_and_tokenize, fn_kwargs={"prompt_start_token": prompt_start_token, "prompt_end_token": prompt_end_token}
    )

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

*****************************************************************************************
57529
57530
<class 'torch.Tensor'> torch.Size([3, 1280, 960])
<class 'torch.Tensor'> torch.Size([128])
tensor([[13]])
tensor([[18]])
*****************************************************************************************
57529
57530
<class 'torch.Tensor'> torch.Size([3, 1280, 960])
<class 'torch.Tensor'> torch.Size([128])
tensor([[14],
        [34]])
tensor([[19],
        [39]])
*****************************************************************************************
57529
57530
<class 'torch.Tensor'> torch.Size([3, 1280, 960])
<class 'torch.Tensor'> torch.Size([128])
tensor([[15]])
tensor([[24]])
*****************************************************************************************
57529
57530
<class 'torch.Tensor'> torch.Size([3, 1280, 960])
<class 'torch.Tensor'> torch.Size([128])
tensor([[12]])
tensor([[19]])
********************************************************************************

In [59]:
# ## For Doc Classification task
# prompt_start_token = "<s_docClass>"
# prompt_end_token = "</s_docClass>"

# hf_sub_processed_final = hf_sub_ocr_gt_prompt.map(
#     transform_and_tokenize, fn_kwargs={"prompt_start_token": prompt_start_token, "prompt_end_token": prompt_end_token}
#     )

In [60]:
# ## For Doc Prsing task
# prompt_start_token = "<s_docParse>"
# prompt_end_token = "</s_docParse>"

# hf_sub_processed_final = hf_sub_ocr_gt_prompt.map(
#     transform_and_tokenize, fn_kwargs={"prompt_start_token": prompt_start_token, "prompt_end_token": prompt_end_token}
#     )

In [61]:
## Converting Huggingface tensor formatting for specific columns using .with_format from Python lists to PyTorch Tensor
hf_sub_processed_final = hf_sub_processed_final.with_format(type="torch", columns=["input_tensor_pixel", "decoder_input_ids", "labels"])

### Note:fn_kwargs
In the context of the `map` method of a Hugging Face dataset, `fn_kwargs` refers to the keyword arguments that you can pass to the mapping function.

The `map` method allows you to apply a mapping function to each example in the dataset. You can provide additional keyword arguments to the mapping function using the `fn_kwargs` parameter. The `fn_kwargs` parameter takes a dictionary where the keys are the argument names and the values are the corresponding argument values.

For example:

```python
def my_mapping_function(example, arg1, arg2):
    # Process the example using arg1 and arg2
    ...

dataset = dataset.map(
    my_mapping_function,
    fn_kwargs={"arg1": value1, "arg2": value2}
)
```

In this example, the `my_mapping_function` is the mapping function that takes additional arguments `arg1` and `arg2`. The `fn_kwargs` parameter is used to pass the values `value1` and `value2` to `arg1` and `arg2`, respectively.

By using `fn_kwargs`, you can pass additional arguments to your mapping function and customize its behavior based on those arguments.

In [62]:
hf_sub_processed_final    # details of final processed huggingface data

Dataset({
    features: ['question', 'docId', 'answers', 'data_split', 'full_path_image', 'bounding_boxes', 'word_list', 'ground_truth', 'gt_prompt', 'input_tensor_pixel', 'decoder_input_ids', 'labels'],
    num_rows: 10
})

#### Sample Check of row 0,1,2

In [63]:
## Sample input tensor pixel check for row 0
hf_sub_processed_final['input_tensor_pixel'][0]

tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]],

        [[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]])

In [64]:
type(hf_sub_processed_final['input_tensor_pixel'][0]), hf_sub_processed_final['input_tensor_pixel'][0].shape

(torch.Tensor, torch.Size([3, 1280, 960]))

In [65]:
## decoder input ids checking for row 0
hf_sub_processed_final['decoder_input_ids'][0], type(hf_sub_processed_final['decoder_input_ids'][0])

(tensor([    0, 57527, 27973, 52743, 48941, 44960, 40053, 35647, 25515, 54964,
         36322, 36209, 57528, 57529,  1314, 27810, 47106,  4050, 57530,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

In [66]:
## labels checking for row 0
hf_sub_processed_final['labels'][0]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  1314, 27810, 47106,  4050,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])

In [67]:
## decoder input ids checking for row 1
hf_sub_processed_final['decoder_input_ids'][1]

tensor([    0, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
        25515, 36322, 36209, 57528, 57529, 40769, 39539, 39473, 49798, 57530,
        57522, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
        25515, 36322, 36209, 57528, 57529, 55285, 39539, 17662, 39905, 57530,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1])

In [68]:
## labels checking for row 1
hf_sub_processed_final['labels'][1]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100, 40769, 39539, 39473, 49798,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100, 55285, 39539, 17662, 39905,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])

In [69]:
## Sample decoder input ids check of row 2
hf_sub_processed_final['decoder_input_ids'][2]

tensor([    0, 57527, 46193, 56931, 22964, 47459,  8888, 28431, 46192, 36322,
        36294, 52743, 54964, 36209, 57528, 57529, 39611, 46762, 39710, 20221,
        49703, 39571,  8342, 53498, 57530,     2,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1])

In [70]:
type(hf_sub_processed_final['decoder_input_ids'][2]), hf_sub_processed_final['decoder_input_ids'][2].shape

(torch.Tensor, torch.Size([128]))

In [71]:
## Sample labels check of row 2
hf_sub_processed_final['labels'][2]

tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100, 39611, 46762, 39710, 20221,
        49703, 39571,  8342, 53498,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100])

In [72]:
type(hf_sub_processed_final['labels'][2]), hf_sub_processed_final['labels'][2].shape

(torch.Tensor, torch.Size([128]))

In [73]:
# from datasets import load_from_disk
# from transformers import DonutProcessor

## COMMENT IN in case you want to save the processed dataset to disk in case of error later
# processed_dataset.save_to_disk("processed_dataset")
# processor.save_pretrained("processor")

## COMMENT IN in case you want to load the processed dataset from disk in case of error later
# processed_dataset = load_from_disk("processed_dataset")
# processor = DonutProcessor.from_pretrained("processor")

## S-5: Converting Huggingface Dataset to PyTorch Dataset

In [74]:
from torch.utils.data import Dataset

In [75]:
## Creating custom class to convert Huggingface to PyTorch Dataset

class DonutDatasetWrapper(Dataset):
  def __init__(self, dataset_name_or_path: str,):

    super().__init__()
    self.hf_dataset = dataset_name_or_path

  def __len__(self):
    return len(self.hf_dataset) -1

  def __getitem__(self, idx):
    # Retrieve the desired columns from the Hugging Face dataset
    input_tensor_pixel = self.hf_dataset[idx]['input_tensor_pixel']
    input_ids = self.hf_dataset[idx]['decoder_input_ids']
    labels = self.hf_dataset[idx]['labels']

    # Return the selected columns as tensors
    return input_tensor_pixel, input_ids, labels


### Note: why len - 1?
In this class, the `__len__()` method is implemented to return the length of the dataset. However, the value returned is `self.dataset_length - 1`. The reason behind subtracting 1 is to account for zero-based indexing.

In Python, indexing starts from 0, so if the dataset has `n` samples, their indices will range from 0 to `n-1`. By subtracting 1 from the dataset length, the `__len__()` method ensures that the index values are within the valid range.

For example, if `self.dataset_length` is 100 (indicating a dataset with 100 samples), the valid indices will be 0 to 99. Therefore, returning `self.dataset_length - 1` from `__len__()` ensures that the dataset length aligns with the valid indices of the dataset.

In [76]:
# Assuming you have the Hugging Face dataset `hf_sub_processed_final`
train_dataset_sub = DonutDatasetWrapper(hf_sub_processed_final)

In [77]:
train_dataset_sub   # details of PyTorch dataset format

In [78]:
len(train_dataset_sub)    # length of train dataset

9

In [79]:
type(train_dataset_sub)

__main__.DonutDatasetWrapper

##### Sample row 0:

In [80]:
train_dataset_sub[0]        ## details of row 0

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([    0, 57527, 27973, 52743, 48941, 44960, 40053, 35647, 25515, 54964,
   

In [81]:
## Sample check of row 0
pixel_values_0, decoder_input_ids_0, labels_0  = train_dataset_sub[0]
for decoder_input_id, label in zip(decoder_input_ids_0.tolist()[:-1], labels_0.tolist()[:]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
date -100
mention -100
ed -100
in -100
this -100
letter -100
? -100
</s_question> -100
<s_answer> -100
1 1
/8 /8
/ /
93 93
</s_answer> -100
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad

##### Sample row 1:

In [82]:
train_dataset_sub[1]        ## details of row 1

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([    0, 57527, 27973, 52743, 48941, 44474, 37517, 55856, 40053, 35647,
   

In [83]:
## Sample check of row 1
pixel_values_1, decoder_input_ids_1, labels_1  = train_dataset_sub[1]
for decoder_input_id, label in zip(decoder_input_ids_1.tolist()[:-1], labels_1.tolist()[:]):
  if label != -100:
    print(processor.decode([decoder_input_id]), processor.decode([label]))
  else:
    print(processor.decode([decoder_input_id]), label)

<s> -100
<s_question> -100
what -100
is -100
the -100
contact -100
person -100
name -100
mention -100
ed -100
in -100
letter -100
? -100
</s_question> -100
<s_answer> -100
P P
. .
Car Car
ter ter
</s_answer> -100
<sep/> -100
<s_question> -100
what -100
is -100
the -100
contact -100
person -100
name -100
mention -100
ed -100
in -100
letter -100
? -100
</s_question> -100
<s_answer> -100
p p
. .
carte carte
r r
</s_answer> -100
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
